In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
model_groq = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=groq_api_key)
model_groq


if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY tidak ditemukan. Pastikan sudah diatur di file .env.")


d:\LIFE\_BOOTCAMP_MANDIRI\B2_GENAI_LANGCHAIN\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain

In [4]:
import bs4

strainer_wikipedia = bs4.SoupStrainer(
    id="mw-content-text"
)

loader = WebBaseLoader(
    web_paths=("https://id.wikipedia.org/wiki/Kecerdasan_buatan",),
    bs_kwargs={"parse_only": strainer_wikipedia}
)

In [5]:
docs = loader.load()


In [6]:
# Ini akan mencetak teks bersih HANYA dari README.md
print(docs[0].page_content)

Artikel ini membutuhkan rujukan tambahan agar kualitasnya dapat dipastikan. Mohon bantu kami mengembangkan artikel ini dengan cara menambahkan rujukan ke sumber tepercaya. Pernyataan tak bersumber bisa saja dipertentangkan dan dihapus.Cari sumber: "Kecerdasan buatan" – berita · surat kabar · buku · cendekiawan · JSTOR
Robot ASIMO menggunakan pengindra dan algoritme kecerdasan buatan untuk menuruni tangga dan menghindari rintangan
Kecerdasan buatan atau akal imitasi[1] (bahasa Inggris: artificial intelligence, AI) adalah kecerdasan yang ditambahkan kepada suatu sistem yang bisa diatur dalam konteks ilmiah, AI juga didefinisikan sebagai kecerdasan entitas ilmiah. Andreas Kaplan dan Michael Haenlein mendefinisikan kecerdasan buatan sebagai "kemampuan sistem untuk menafsirkan data eksternal dengan benar, untuk belajar dari data tersebut, dan menggunakan pembelajaran tersebut guna mencapai tujuan dan tugas tertentu melalui adaptasi yang luwes".[2] Sistem seperti ini umumnya dianggap kompute

In [7]:
if docs:
    original_text = docs[0].page_content
    
    # Pisahkan teks menjadi kata-kata
    words = original_text.split()
    
    # Ambil 500 kata pertama
    first_500_words_list = words[:400]
    
    # Gabungkan kembali menjadi satu string
    limited_text = " ".join(first_500_words_list)
    
    # Anda bisa perbarui dokumennya jika mau
    docs[0].page_content = limited_text
    
    print("\n--- 500 KATA PERTAMA ---")
    print(docs[0].page_content)
    print(f"\n... (Total kata: {len(first_500_words_list)})")

else:
    print("Gagal memuat dokumen.")


--- 500 KATA PERTAMA ---
Artikel ini membutuhkan rujukan tambahan agar kualitasnya dapat dipastikan. Mohon bantu kami mengembangkan artikel ini dengan cara menambahkan rujukan ke sumber tepercaya. Pernyataan tak bersumber bisa saja dipertentangkan dan dihapus.Cari sumber: "Kecerdasan buatan" – berita · surat kabar · buku · cendekiawan · JSTOR Robot ASIMO menggunakan pengindra dan algoritme kecerdasan buatan untuk menuruni tangga dan menghindari rintangan Kecerdasan buatan atau akal imitasi[1] (bahasa Inggris: artificial intelligence, AI) adalah kecerdasan yang ditambahkan kepada suatu sistem yang bisa diatur dalam konteks ilmiah, AI juga didefinisikan sebagai kecerdasan entitas ilmiah. Andreas Kaplan dan Michael Haenlein mendefinisikan kecerdasan buatan sebagai "kemampuan sistem untuk menafsirkan data eksternal dengan benar, untuk belajar dari data tersebut, dan menggunakan pembelajaran tersebut guna mencapai tujuan dan tugas tertentu melalui adaptasi yang luwes".[2] Sistem seperti in

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vectorstore.as_retriever()
retriever


VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000029D44A73AC0>, search_kwargs={})

In [9]:
# Prompt Template
system_prompt = ("""Gunakan konteks berikut untuk menjawab pertanyaan di akhir.
Jika jawaban tidak ada di dalam konteks, katakan "Saya tidak tahu."

Konteks:
{context}

""")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [10]:
question_answer_chain = create_stuff_documents_chain(model_groq, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [11]:
response = rag_chain.invoke({"input": "apa itu kecerdan buatan"})

In [12]:
response['answer']

'Kecerdasan buatan (bahasa Inggris: artificial intelligence, AI) adalah kecerdasan yang ditambahkan kepada suatu sistem yang bisa diatur dalam konteks ilmiah. AI juga didefinisikan sebagai kecerdasan entitas ilmiah.\n\nAndreas Kaplan dan Michael Haenlein mendefinisikan kecerdasan buatan sebagai "kemampuan sistem untuk menafsirkan data eksternal dengan benar, untuk belajar dari data tersebut, dan menggunakan pembelajaran tersebut guna mencapai tujuan dan tugas tertentu melalui adaptasi yang luwes".\n\nSistem seperti ini umumnya dianggap memiliki kemampuan seperti:\n\n* Pengenalan tulisan tangan, suara, dan wajah\n* Pembelajaran dari data\n* Adaptasi terhadap situasi yang berbeda\n* Membuat keputusan berdasarkan data\n* Menguasai permainan catur dan lain-lain\n\nKecerdasan buatan ini bukan hanya ingin mengerti apa itu sistem kecerdasan, tetapi juga membangunnya. Tidak ada definisi yang memuaskan untuk \'kecerdasan\': kecerdasan: kemampuan untuk memperoleh pengetahuan dan menggunakannya a

In [17]:
## Adding chat hitory
response1 = rag_chain.invoke({"input":"bagaimana cara kerjanya?"})

In [18]:
response1['answer']

'Kecerdasan buatan (AI) bekerja dengan menggunakan beberapa komponen utama, yaitu:\n\n1. **Penginderaan (Sensors)**: Penginderaan adalah komponen yang mengumpulkan data dari lingkungan sekitar, seperti sensor suhu, sensor cahaya, dan lain-lain.\n2. **Algoritme**: Algoritme adalah komponen yang mengolah data yang dikumpulkan oleh penginderaan untuk menemukan pola, membuat keputusan, dan melakukan tindakan.\n3. **Pemrosesan (Processing)**: Pemrosesan adalah komponen yang mengolah data yang telah dikumpulkan dan diolah oleh algoritme untuk membuat keputusan atau melakukan tindakan.\n4. **Memori (Memory)**: Memori adalah komponen yang menyimpan data yang telah dikumpulkan, diolah, dan dihasilkan oleh sistem AI.\n5. **Kontrol (Controller)**: Kontrol adalah komponen yang mengontrol aliran data dan proses dalam sistem AI.\n\nCara kerja AI dapat dibagi menjadi beberapa tahap, yaitu:\n\n1. **Pengumpulan Data**: Sistem AI mengumpulkan data dari lingkungan sekitar melalui penginderaan.\n2. **Peng

In [ ]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

conntextualize_q_system__prompts = """Anda adalah asisten AI yang cerdas dan dapat mengingat percakapan.
Tugas Anda adalah melanjutkan dialog dengan pengguna secara alami.

Gunakan **Riwayat Percakapan** sebelumnya sebagai konteks utama.
Beberapa pertanyaan mungkin merujuk ke topik di riwayat (misal: "bagaimana dengan yang tadi?", "jelaskan lebih lanjut").

Jawab **Pertanyaan Terbaru Pengguna** dengan relevan, ringkas, dan sopan, seolah-olah Anda melanjutkan obrolan yang sedang berlangsung.

**Riwayat Percakapan:**
{chat_history}

**Pertanyaan Terbaru Pengguna:**
{input}

**Jawaban Anda:**
"""